In [10]:
import os
import time
import pandas as pd
from sqlalchemy import create_engine
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Configuración de la base de datos
database_config = {
    'server': 'dislicores-1.cxc0gcgy4dsz.us-east-2.rds.amazonaws.com,1433',
    'database': 'dislicores-1',
    'username': 'admin',
    'password': 'KingKong9#_9+',
}

# Crear cadena de conexión
connection_string = f"mssql+pyodbc://{database_config['username']}:{database_config['password']}@{database_config['server']}/{database_config['database']}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

# Verificación de conexión
try:
    engine.connect()
    print("Conexión exitosa a la base de datos")
except Exception as e:
    print(f"Error en la conexión: {e}")

class DataHandler(FileSystemEventHandler):
    def __init__(self, engine):
        self.engine = engine

    def on_created(self, event):
        if event.is_directory or not event.src_path.endswith(".csv"):
            return
        self.process_new_file(event.src_path)

    def on_modified(self, event):
        if event.is_directory or not event.src_path.endswith(".csv"):
            return
        self.process_new_file(event.src_path)

    def process_new_file(self, file_path):
        print(f"Procesando el archivo: {file_path}")
        try:
            new_data = pd.read_csv(file_path)
            print(f"Datos cargados desde {file_path}")

            new_data = new_data.drop(columns=['Unnamed: 0'], errors='ignore')

            # Convertir columnas que contengan la palabra 'date' a formato datetime
            for column in new_data.columns:
                if 'date' in column.lower():
                    new_data[column] = pd.to_datetime(new_data[column], errors='coerce')

            table_name = os.path.splitext(os.path.basename(file_path))[0]
            print(f"Tabla detectada: {table_name}")

            # Obtener el nombre de la columna ID específica
            id_column = self.get_id_column(table_name)

            # Verificar que la columna ID exista en el DataFrame
            if id_column not in new_data.columns:
                print(f"Error: La columna '{id_column}' no existe en el archivo {file_path}")
                return

            # Cargar los datos existentes de la base de datos
            print(f"Cargando datos existentes de la tabla {table_name} desde la base de datos...")
            existing_data = pd.read_sql(f"SELECT * FROM {table_name}", self.engine)
            print(f"Datos existentes cargados de la tabla {table_name}")

            # Identificar filas nuevas (basado en id_column)
            new_rows = new_data[~new_data[id_column].isin(existing_data[id_column])]

            # Imprimir las filas nuevas detectadas
            print("Filas nuevas identificadas:")
            print(new_rows)

            # Insertar datos nuevos en la base de datos
            if not new_rows.empty:
                print(f"Insertando {len(new_rows)} filas nuevas en la tabla {table_name}...")
                new_rows.to_sql(table_name, self.engine, if_exists='append', index=False)
                print(f"Se han insertado {len(new_rows)} filas nuevas en la tabla {table_name}")
            else:
                print(f"No hay filas nuevas para insertar en la tabla {table_name}")

        except Exception as e:
            print(f"Error al procesar el archivo {file_path}: {e}")

    def get_id_column(self, table_name):
        id_columns = {
            'VentasFinales': 'VentasID',
            'DetalleCompra': 'DetalleCompraID',
            'InventarioFinal2016': 'InventoryFinalID',
            'InventarioInicial2016': 'InventoryInicialID',
            'Producto': 'BrandID',
            'FacturaCompras': 'CompraID',
        }
        return id_columns.get(table_name, 'id')

def process_existing_files(path, handler):
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            file_path = os.path.join(path, filename)
            handler.process_new_file(file_path)

if __name__ == '__main__':
    path_to_watch = r"C:\Users\Juan Daniel Bermudez\OneDrive\Escritorio\M5\NuevaData"  # Cambia esto por tu ruta
    event_handler = DataHandler(engine)

    # Procesar archivos existentes en la carpeta
    print("Procesando archivos existentes en la carpeta...")
    process_existing_files(path_to_watch, event_handler)

    observer = Observer()
    observer.schedule(event_handler, path=path_to_watch, recursive=True)
    observer.start()
    print(f"Observando cambios en: {path_to_watch}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()
    print("El observador ha sido detenido.")




Conexión exitosa a la base de datos
Procesando archivos existentes en la carpeta...
Procesando el archivo: C:\Users\Juan Daniel Bermudez\OneDrive\Escritorio\M5\NuevaData\VentasFinales.csv
Datos cargados desde C:\Users\Juan Daniel Bermudez\OneDrive\Escritorio\M5\NuevaData\VentasFinales.csv
Tabla detectada: VentasFinales
Cargando datos existentes de la tabla VentasFinales desde la base de datos...
Datos existentes cargados de la tabla VentasFinales
Filas nuevas identificadas:
  InventoryID  Store  BrandID     Description   Size  SalesQuantity  \
0      winter     19    39745  Whisky 12 años  700ml             10   

   SalesDollars  SalesPrice  SalesDate  Volume  Classification  ExciseTax  \
0         459.9       45.99 2023-05-01     700               2       0.22   

   VendorNo              VendorName  InventoryInicialID  VentasID  Cantidad  
0      2000  Distribuidora Nacional             1048576   1048576        50  
Insertando 1 filas nuevas en la tabla VentasFinales...
Se han inser